In [1]:
import pandas as pd
import os
import re


#Nous ouvrons le dataset
df_ACE = pd.read_csv(os.path.join(r"comptages-routiers-permanents_ACE.csv"), sep=";")
df_Sts = pd.read_csv(os.path.join(r"comptages-routiers-permanents_Sts.csv"), sep=";")
df_convention = pd.read_csv(os.path.join(r"comptages-routiers-permanents_convention.csv"), sep=";")


#print(df_ACE.head())

(df_ACE, df_Sts, df_convention)
tagg = ["Champs Elysées", "Sts-Pères", "Convention"]

def clean_date(date):
    
    date = re.sub('T', ' ', date)
    date=date[:-6]
    return date

#compte_1=df_ACE.groupby("Etat arc")["Date et heure de comptage"].nunique()
for df in (df_ACE, df_Sts, df_convention):
    df['Date et heure de comptage'] = df['Date et heure de comptage'].apply(lambda s : clean_date(s))

#df_convention['Date et heure de comptage'] = df_convention['Date et heure de comptage'].apply(lambda s : clean_date(s))
#df_Sts['Date et heure de comptage'] = df_Sts['Date et heure de comptage'].apply(lambda s : clean_date(s))

#Nous adaptons notre features "Date" au format d'une série temporelle

for df in (df_ACE, df_Sts, df_convention):
    df['Date et heure de comptage']= pd.to_datetime(df["Date et heure de comptage"], format='%Y-%m-%d %H:%M:%S')
 
    
for df in (df_ACE, df_Sts, df_convention):
    df.sort_values("Date et heure de comptage",inplace = True)
    print(df.head())

df_rues = pd.concat([df_ACE, df_Sts, df_convention])
#Nous indiquons que la série temporelle est indexée selon la date


#df_rues = df_ACE.set_index('Date et heure de comptage')

    
print(df_rues.index)

#Nous créons plusieurs colonnes permettant de situer la donnée par jour/mois/année. Cela nous servira pa la suite pour traiter
#la données selon différentes échelles temporelles.


#df_rues['Year'] = df_rues.index.year
#df_rues['Month'] = df_rues.index.month
#df_rues['Weekday Name'] = df_rues.index.weekday
#df_rues['Hour'] = df_rues.index.hour



print(df_rues.head())

      Identifiant arc            Libelle Date et heure de comptage  \
6712             4264  AV_Champs_Elysees       2019-11-01 04:00:00   
6713             4264  AV_Champs_Elysees       2019-11-01 05:00:00   
6714             4264  AV_Champs_Elysees       2019-11-01 06:00:00   
2607             4264  AV_Champs_Elysees       2019-11-01 07:00:00   
2608             4264  AV_Champs_Elysees       2019-11-01 08:00:00   

      Débit horaire  Taux d'occupation Etat trafic  Identifiant noeud amont  \
6712          746.0           10.98889      Fluide                     2294   
6713          549.0            7.40722      Fluide                     2294   
6714          503.0            7.66667      Fluide                     2294   
2607          500.0            4.75500      Fluide                     2294   
2608          525.0            5.37389      Fluide                     2294   

               Libelle noeud amont  Identifiant noeud aval  \
6712  Av_Champs_Elysees-Washington        

In [2]:

df_rues.drop(columns=["Libelle noeud aval", "Libelle noeud amont", "Libelle", "Identifiant noeud aval", "Identifiant noeud amont", "geo_point_2d", "geo_shape", "Date debut dispo data", "Date debut dispo data"], inplace = True)
print(df_rues.head())
compte_1=df_rues.groupby("Identifiant arc").nunique()
print(compte_1)

      Identifiant arc Date et heure de comptage  Débit horaire  \
6712             4264       2019-11-01 04:00:00          746.0   
6713             4264       2019-11-01 05:00:00          549.0   
6714             4264       2019-11-01 06:00:00          503.0   
2607             4264       2019-11-01 07:00:00          500.0   
2608             4264       2019-11-01 08:00:00          525.0   

      Taux d'occupation Etat trafic  Etat arc Date fin dispo data  
6712           10.98889      Fluide  Invalide          2019-06-01  
6713            7.40722      Fluide  Invalide          2019-06-01  
6714            7.66667      Fluide  Invalide          2019-06-01  
2607            4.75500      Fluide  Invalide          2019-06-01  
2608            5.37389      Fluide  Invalide          2019-06-01  
                 Date et heure de comptage  Débit horaire  Taux d'occupation  \
Identifiant arc                                                                
191                                

In [3]:
def chunks(l, size, step):
    """ Yield successive n-sized chunks from l.
    """
    l = l[:(len(l) // size) * size]
    return [x for x in [l[i : i + size] for i in range(0, len(l), step)] if len(x) == size]

In [ ]:
df = df_rues

df = df.replace({4264:"1", 5671: "2", 191:"3"})
df["Identifiant arc"]=df["Identifiant arc"].astype(str)
print(df.head())
print(df.dtypes)

In [110]:
#paramètres
N_input = 7*24
N_output = 1*24
device = 'cuda'
p_test = 0.2
optimizer_parameters = {'lr': 0.001}
batch_size = 6

In [111]:
#Nous précisons ici que la série temporelle possède une fréquence à l'heure et nous remplaçons les valeurs manquantes par
#la valeur qui les précèdent.

missing_df_1= df[df.isnull().any(1)]
print(missing_df_1)
df = df.dropna()

Empty DataFrame
Columns: [Identifiant arc, Date et heure de comptage, Débit horaire, Taux d'occupation, Etat trafic, Etat arc, Date fin dispo data]
Index: []


In [112]:
import numpy as np
#df_meta = df["Service"]
#df_meta['Service'] = df_meta.index.astype(str)
#df_meta.astype(pd.CategoricalDtype(categories=df_meta.index)).cat.codes
#print(df_meta.head())
df["Débit horaire"]= df["Débit horaire"].astype(int)
data = df.sort_values(['Date et heure de comptage', 'Identifiant arc']).groupby('Identifiant arc')["Débit horaire"].apply(lambda x: chunks(x.values, size=N_input+N_output, step=N_output))
#print(data.head())

#print(data.values)
data_numpy = np.array([np.array(x) for x in data.values])

print(data_numpy.dtype)
dep = np.array([x for x in data.index])
#print(dep)

object


d:\anaconda\envs\ml_mvp_1\lib\site-packages\ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


In [113]:
# Mélange les services
ind = np.arange(len(data_numpy))
np.random.shuffle(ind)
data_numpy = data_numpy[ind]

In [114]:
# Sépare en train/test
nb_train = int(len(data_numpy)*(1-p_test))

data_numpy_test = data_numpy[nb_train:]
data_numpy_train = data_numpy[:nb_train]

In [115]:
# Concatène chaque départements ensembles
data_numpy_train = np.concatenate(data_numpy_train)
data_numpy_test = np.concatenate(data_numpy_test)

In [116]:
# Normalise par mean-std calculées sur le data de train
print(data_numpy_train)
std = data_numpy_train.std((0,1))[None]
mean = data_numpy_train.mean((0,1))[None]
data_numpy_train = (data_numpy_train-mean[None])/std[None]
data_numpy_test = (data_numpy_test-mean[None])/std[None]
print(data_numpy_train)
def rescale(data):
    return data * std + mean

[[746 549 503 ... 879 745 541]
 [466 393 434 ... 798 682 574]
 [679 463 488 ... 727 625 445]
 ...
 [296 273 229 ... 359 410 443]
 [276 221 182 ... 594 568 557]
 [462 391 279 ... 659 668 679]]
[[ 0.50706914 -0.00186829 -0.12070647 ...  0.85066649  0.5044857
  -0.0225358 ]
 [-0.2162937  -0.40488473 -0.29896374 ...  0.64140795  0.34172906
   0.06271768]
 [ 0.33397874 -0.22404402 -0.15945805 ...  0.4579838   0.19447305
  -0.27054592]
 ...
 [-0.65547828 -0.71489737 -0.82856868 ... -0.49272165 -0.36096627
  -0.27571279]
 [-0.70714706 -0.84923619 -0.9499903  ...  0.11438645  0.04721705
   0.01879922]
 [-0.22662746 -0.41005161 -0.69939674 ...  0.28230997  0.30556092
   0.33397874]]


In [117]:
# Sépare en input/output
X_train, y_train = data_numpy_train[:, :N_input], data_numpy_train[:, N_input:]
X_test, y_test = data_numpy_test[:, :N_input], data_numpy_test[:, N_input:]
print(X_train)

[[ 0.50706914 -0.00186829 -0.12070647 ...  0.29522716  0.01879922
  -0.51080572]
 [-0.2162937  -0.40488473 -0.29896374 ...  0.85066649  0.5044857
  -0.0225358 ]
 [ 0.33397874 -0.22404402 -0.15945805 ...  0.64140795  0.34172906
   0.06271768]
 ...
 [-0.65547828 -0.71489737 -0.82856868 ... -0.04578675 -0.05870394
  -0.14137398]
 [-0.70714706 -0.84923619 -0.9499903  ... -0.49272165 -0.36096627
  -0.27571279]
 [-0.22662746 -0.41005161 -0.69939674 ...  0.11438645  0.04721705
   0.01879922]]


In [118]:
# Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install torch===1.6.0 torchvision===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html
import torch

#import torch
print(type(X_train))
print(data_numpy.dtype)
print(torch.from_numpy(X_train))
dataset_train = torch.utils.data.TensorDataset(torch.FloatTensor(X_train[:,:,None]),
                                               torch.FloatTensor(y_train[:,:,None]))
dataset_test = torch.utils.data.TensorDataset(torch.FloatTensor(X_test[:,:,None]),
                                              torch.FloatTensor(y_test[:,:,None]))

trainloader = torch.utils.data.DataLoader(
    dataset_train, batch_size=batch_size, shuffle=True, num_workers=1)
testloader = torch.utils.data.DataLoader(
    dataset_test, batch_size=batch_size, shuffle=False, num_workers=1)

<class 'numpy.ndarray'>
object
tensor([[ 0.5071, -0.0019, -0.1207,  ...,  0.2952,  0.0188, -0.5108],
        [-0.2163, -0.4049, -0.2990,  ...,  0.8507,  0.5045, -0.0225],
        [ 0.3340, -0.2240, -0.1595,  ...,  0.6414,  0.3417,  0.0627],
        ...,
        [-0.6555, -0.7149, -0.8286,  ..., -0.0458, -0.0587, -0.1414],
        [-0.7071, -0.8492, -0.9500,  ..., -0.4927, -0.3610, -0.2757],
        [-0.2266, -0.4101, -0.6994,  ...,  0.1144,  0.0472,  0.0188]],
       dtype=torch.float64)


In [119]:
from tslearn.metrics import dtw, dtw_path
from tslearn.barycenters import dtw_barycenter_averaging
def fake_baye(outputs, log_sigmas, target):
    loss = 1/2 * (log_sigmas + torch.exp(-log_sigmas) * (outputs - target)**2)
    return loss.mean()

def loss_rescale_factor(log_rescale_factor, outputs, log_sigmas, target):
    batch_size = outputs.shape[0]
    loss = 1/2 * torch.exp(-2 * log_rescale_factor) * torch.sum(torch.exp(-log_sigmas) * (outputs - target)**2)
    loss += batch_size * log_rescale_factor
    return loss.sum()

In [120]:
import tqdm

def train_model(net, loss_type, optimizer_parameters, epochs=1000, gamma=0.001,
                eval_every=50, Lambda=1, alpha=0.5, beta=0.):
    optimizer = torch.optim.Adam(net.parameters(), **optimizer_parameters)
    criterion = torch.nn.MSELoss()

    if hasattr(net, 'log_rescale_factor'):
        optimizer_rescale_factor = torch.optim.Adam([net.log_rescale_factor], **optimizer_parameters)

    t = tqdm.notebook.tqdm(range(epochs))
    for epoch in t:
        for i, data in enumerate(trainloader, 0):

            net.train()
            if hasattr(net, 'log_rescale_factor'):
                net.log_rescale_factor.require_grad = False

            inputs, target = data
            inputs = inputs.to(device)
            target = target.to(device)
            batch_size, N_output = target.shape[0:2]

            # forward + backward + optimize
            outputs = net(inputs)
            if len(outputs) == 2:
                outputs, _ = outputs
            else:
                outputs, log_sigmas, _ = outputs

            loss_mse, loss_shape, loss_temporal = torch.tensor(
                0), torch.tensor(0), torch.tensor(0)

            loss = 0
            if 'mse' in loss_type:
                loss_mse = 100 * criterion(outputs, target)
                loss += loss_mse

            if 'dilate' in loss_type:
                loss_dilate, loss_shape, loss_temporal = dilate_loss(
                    outputs, target, alpha, gamma, device)
                loss += loss_dilate

            if 'fake_baye' in loss_type:
                loss_fake_baye = fake_baye(outputs, log_sigmas, target)
                loss += loss_fake_baye

            if 'all_target_regression' in loss_type:
                loss_target_week_regression = F.mse_loss(outputs.sum(1), target.sum(1))
                loss += loss_target_week_regression

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if hasattr(net, 'log_rescale_factor'):
                net.log_rescale_factor.require_grad = True

                loss_rescale = loss_rescale_factor(net.log_rescale_factor, outputs.detach(), log_sigmas.detach(),
                                                   target)
                optimizer_rescale_factor.zero_grad()
                loss_rescale.backward()
                optimizer_rescale_factor.step()

        if (epoch % eval_every == 0):
            eval_mse, eval_dtw, eval_tdi = eval_model(net, testloader, gamma)

        t.set_postfix(loss=loss.item(),
                      loss_shape=loss_shape.item(),
                      loss_temporal=loss_temporal.item(),
                      mse=eval_mse,
                      dtw=eval_dtw,
                      tdi=eval_tdi)

In [121]:
def eval_model(net, loader, gamma):
    criterion = torch.nn.MSELoss()
    losses_mse = []
    losses_dtw = []
    losses_tdi = []

    net.eval()

    for i, data in enumerate(loader, 0):
        loss_mse, loss_dtw, loss_tdi = torch.tensor(
            0), torch.tensor(0), torch.tensor(0)
        # get the inputs
        inputs, target = data
        inputs = inputs.to(device)
        target = target.to(device)
        batch_size, N_output = target.shape[0:2]
        outputs = net(inputs)
        if len(outputs) == 2:
            outputs, _ = outputs
        else:
            outputs, prewarp_outputs, warp_matrix = outputs

        # MSE
        loss_mse = criterion(target, outputs)
        loss_dtw, loss_tdi = 0, 0
        # DTW and TDI
        for k in range(batch_size):
            target_k_cpu = target[k, :, 0:1].view(-1).detach().cpu().numpy()
            output_k_cpu = outputs[k, :, 0:1].view(-1).detach().cpu().numpy()

            loss_dtw += dtw(target_k_cpu, output_k_cpu)
            path, sim = dtw_path(target_k_cpu, output_k_cpu)

            Dist = 0
            for i, j in path:
                Dist += (i - j) * (i - j)
            loss_tdi += Dist / (N_output * N_output)

        loss_dtw = loss_dtw / batch_size
        loss_tdi = loss_tdi / batch_size

        # print statistics
        losses_mse.append(loss_mse.item())
        losses_dtw.append(loss_dtw)
        losses_tdi.append(loss_tdi)

    return np.array(losses_mse).mean(), np.array(losses_dtw).mean(), np.array(losses_tdi).mean()


In [122]:
import torch.nn as nn
import torch.nn.functional as F

class Encoder(nn.Module):
    def __init__(self, input_dim, enc_hid_dim, dec_hid_dim, num_layers, dropout):
        super().__init__()
        self.enc_hid_dim = enc_hid_dim
        self.num_layers = num_layers
        self.rnn = nn.GRU(input_dim, enc_hid_dim,
                          num_layers=num_layers,
                          bidirectional=True,
                          batch_first=True,
                          dropout=dropout)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, src, hidden=None):

        outputs, hidden = self.rnn(src, hidden)

        # outputs = [batch size, src len, hid dim * num directions]
        # hidden = [n layers * num directions, batch size, hid dim]

        hidden = torch.cat((hidden[::2, :, :], hidden[1::2, :, :]), dim=2)
        hidden = self.dropout(hidden)
        hidden = torch.tanh(self.fc(hidden))

        # outputs = [batch size, src len, enc hid dim * 2]
        # hidden = [batch size, dec hid dim]

        return outputs, hidden


class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()

        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):

        # hidden = [batch size, dec hid dim]
        # encoder_outputs = [batch size, src len, enc hid dim * 2]

        src_len = encoder_outputs.shape[1]

        # repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)

        # hidden = [batch size, src len, dec hid dim]
        # encoder_outputs = [batch size, src len, enc hid dim * 2]

        energy = torch.tanh(
            self.attn(torch.cat((hidden, encoder_outputs), dim=2)))

        # energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)

        # attention= [batch size, src len]

        return F.softmax(attention, dim=1)


class Decoder(nn.Module):
    def __init__(self, output_dim, enc_hid_dim, dec_hid_dim, fc_dim,
                 num_layers, dropout, attention, bayesian=False, embed_dim=None):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        self.num_layers = num_layers
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.fc_dim = fc_dim
        
        self.embed_dim = embed_dim if embed_dim is not None else 0
        
        self.bayesian = bayesian
        
        if self.bayesian:
            self.out_log_sigma = nn.Linear(fc_dim + output_dim, output_dim)

            self.rnn = nn.GRU((enc_hid_dim * 2) + (output_dim * 2),
                              dec_hid_dim,
                              num_layers=num_layers,
                              batch_first=True,
                              dropout=dropout)
            self.fc = nn.Linear(
                (enc_hid_dim * 2) + dec_hid_dim + (output_dim * 2) + self.embed_dim, fc_dim)

        else:
            self.rnn = nn.GRU((enc_hid_dim * 2) + output_dim,
                              dec_hid_dim,
                              num_layers=num_layers,
                              batch_first=True,
                              dropout=dropout)

            self.fc = nn.Linear(
                (enc_hid_dim * 2) + dec_hid_dim + output_dim + self.embed_dim, fc_dim)

        self.out = nn.Linear(fc_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs, hidden, encoder_outputs, embeddings=None):

        a = self.attention(hidden[-1, :, :], encoder_outputs)

        # a = [batch size, src len]

        a = a.unsqueeze(1)

        # a = [batch size, 1, src len]

        weighted = torch.bmm(a, encoder_outputs)

        # weighted = [batch size, 1, enc hid dim * 2]

        rnn_input = torch.cat((inputs, weighted), dim=2)

        # rnn_input = [batch size, 1, (enc hid dim * 2) + emb dim]

        output, hidden = self.rnn(rnn_input, hidden)

        # output = [batch size, seq len, dec hid dim * n directions]
        # hidden = [n layers * n directions, batch size, dec hid dim]

        hidden_prediction = torch.cat((output, weighted, inputs), dim=2)
        
        if embeddings is not None:
            hidden_prediction = torch.cat((hidden_prediction, embeddings), dim=2)
        
        hidden_prediction = self.fc(hidden_prediction)

        prediction = self.out(F.relu(hidden_prediction))

        # prediction = [batch size, output dim]

        if not self.bayesian:
            return prediction, hidden
        else:
            log_sigma = self.out_log_sigma(torch.cat([hidden_prediction, prediction], dim=-1))
            return prediction, log_sigma, hidden


class MetaEmbedder(nn.Module):
    def __init__(self, cat_input_dim, num_input_dim, embed_dim):
        """cat_input_dim should be a tuple (nb_of_inputs, nb_of_categories)"""
        super().__init__()

        self.cat_input_dim = cat_input_dim
        self.num_input_dim = num_input_dim
        self.embed_dim = embed_dim
        # self.cat_module = nn.Embedding(cat_input_dim[1], embed_dim)
        self.num_module = nn.Linear(num_input_dim, embed_dim)
        #self.fc = nn.Linear(self.embed_dim * (cat_input_dim[0] + 1), self.enc_hid_dim * self.num_layers * 2)
        self.fc = nn.Linear(self.embed_dim, self.embed_dim)

    def forward(self, inputs_cat, inputs_num):
        batch_size = inputs_cat.shape[0]
        #x_cat = self.cat_module(inputs_cat).reshape(batch_size, -1)
        x_num = F.relu(self.num_module(inputs_num))
        #x = torch.cat([x_cat, x_num], dim=1)
        x = x_num
        batch_size = x.shape[0]
        out = self.fc(x)
        return out

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, target_size, embedder=None, mc_dropout=False):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        if not torch.cuda.is_available() and self.device is None: 
            self.device = -1
        self.target_size = target_size
        
        self.bayesian = self.decoder.bayesian
        self.mc_dropout = mc_dropout
        
        if embedder is not None:
            self.embedder = embedder
        
        if self.bayesian:
            #Add a learnable rescale factor as in https://openreview.net/pdf?id=CecZ_0t79q
            self.log_rescale_factor = nn.Parameter(torch.ones(1, requires_grad=True))

    def forward(self, src, meta_cat=None, meta_num=None):
        batch_size = src.shape[0]

        encoder_outputs, hidden = self.encoder(src)
        
        embeddings = None
        if hasattr(self, 'embedder') and meta_cat is not None and meta_num is not None:
            embeddings = self.embedder(meta_cat, meta_num).unsqueeze(1)

        # tensor to store decoder outputs
        outputs = torch.zeros(batch_size,
                              self.target_size,
                              self.decoder.output_dim).to(self.device)
        hiddens = torch.zeros(batch_size,
                              self.target_size,
                              self.decoder.dec_hid_dim).to(self.device)
        
        if self.bayesian:
            log_sigmas = torch.zeros(batch_size,
                                     self.target_size,
                                     self.decoder.output_dim).to(self.device)

        inputs = src[:, -1, :].unsqueeze(1)
        
        if self.bayesian:
            inputs = torch.cat([inputs, torch.zeros(*inputs.shape).to(inputs.device)], 2)

        for t in range(0, self.target_size):
            dec_outputs = self.decoder(inputs, hidden, encoder_outputs, embeddings=embeddings)

            if not self.bayesian:
                output, hidden = dec_outputs
                outputs[:, t:t + 1, :] = output
                hiddens[:, t:t + 1, :] = hidden[-2:-1, :, :].transpose(0, 1)

                inputs = output
            else:
                output, log_sigma, hidden = dec_outputs
                outputs[:, t:t + 1, :] = output
                hiddens[:, t:t + 1, :] = hidden[-2:-1, :, :].transpose(0, 1)
                log_sigmas[:, t:t+1, :] = log_sigma

                inputs = torch.cat([output, log_sigma], dim=2)
        
        #if hasattr(self, 'rescale_factor'):
        #    log_sigmas += 2 * self.rescale_factor

        if not self.bayesian:
            return outputs, hiddens
        else:
            return outputs, log_sigmas, hiddens

In [123]:
def gen_net(dropout=0.5, bayesian=False, mc_dropout=False):
    embedder = None
    encoder = Encoder(input_dim= 1, enc_hid_dim=32, dec_hid_dim=32,
                      num_layers=4, dropout=dropout
                     ).to(device)
    attention = Attention(enc_hid_dim=32, dec_hid_dim=32)
    decoder = Decoder(output_dim= 1, enc_hid_dim=32, dec_hid_dim=32, fc_dim=32,
                      num_layers=4, dropout=dropout, attention=attention, bayesian=bayesian,
                      embed_dim = None
                     ).to(device)
    net = Seq2Seq(encoder, decoder, device, N_output, embedder=embedder, mc_dropout=False).to(device)

    return net

In [124]:
print(torch.cuda.device_count())  
print(torch.cuda.is_available())   
print(torch.version.cuda) 
nets_lists = {}
nets_lists['mse_ensembling'] = []
for i in range(5):
    net = gen_net(dropout=0.2, bayesian=True, mc_dropout=False)
    train_model(net, loss_type=['fake_baye'], optimizer_parameters=optimizer_parameters,
            epochs=251, eval_every=10)
    nets_lists['mse_ensembling'].extend([net])

[x.log_rescale_factor for x in nets_lists['mse_ensembling']]

1
True
10.2


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 2.59 GiB already allocated; 11.16 MiB free; 2.79 GiB reserved in total by PyTorch)